In [ ]:
# Check and install required packages (if not already available)
import subprocess
import sys
import os
from pathlib import Path

def check_and_install_package(package_name, import_name=None):
    """Check if package is installed, install if not"""
    if import_name is None:
        import_name = package_name
    try:
        __import__(import_name)
        print(f"✓ {package_name} is already installed")
        return True
    except ImportError:
        print(f"⚠ {package_name} not found, installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--user", package_name])
        return False

# Check for required packages
required_packages = [
    ("torch", "torch"),
    ("torchvision", "torchvision"),
    ("timm", "timm"),
    ("albumentations", "albumentations"),
    ("pillow", "PIL"),
    ("numpy", "numpy"),
    ("scikit-learn", "sklearn"),
    ("pandas", "pandas"),
    ("tqdm", "tqdm"),
    ("requests", "requests"),
    ("datasets", "datasets"),
    ("psutil", "psutil"),  # For system memory info
    ("peft", "peft"),  # For LoRA training
    ("transformers", "transformers"),  # Required by peft
]

for pkg, imp in required_packages:
    check_and_install_package(pkg, imp)

# Verify GPU availability and system info
import torch
print("\n" + "="*60)
print("SYSTEM INFORMATION")
print("="*60)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU: {gpu_name}")
    print(f"CUDA version: {torch.version.cuda}")
    gpu_props = torch.cuda.get_device_properties(0)
    print(f"GPU Memory: {gpu_props.total_memory / 1e9:.2f} GB")
    print(f"GPU Compute Capability: {gpu_props.major}.{gpu_props.minor}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    
    # RTX 8000 specific optimizations
    if "RTX 8000" in gpu_name or "8000" in gpu_name:
        print("\n✓ RTX 8000 detected - Optimizing for 48GB VRAM")
        # RTX 8000 has 48GB VRAM, can handle larger batch sizes
        os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
else:
    print("⚠ No GPU detected - will use CPU")

# Check available memory
import psutil
mem = psutil.virtual_memory()
print(f"\nSystem RAM: {mem.total / 1e9:.2f} GB")
print(f"Available RAM: {mem.available / 1e9:.2f} GB")

# Check disk space
print("\n" + "="*60)
print("DISK SPACE CHECK")
print("="*60)
disk = psutil.disk_usage('/')
print(f"Total disk: {disk.total / 1e9:.2f} GB")
print(f"Used disk: {disk.used / 1e9:.2f} GB")
print(f"Free disk: {disk.free / 1e9:.2f} GB")
print(f"Usage: {disk.percent:.1f}%")

# Check home directory space
home_disk = psutil.disk_usage(str(Path.home()))
print(f"\nHome directory:")
print(f"  Total: {home_disk.total / 1e9:.2f} GB")
print(f"  Used: {home_disk.used / 1e9:.2f} GB")
print(f"  Free: {home_disk.free / 1e9:.2f} GB")
print(f"  Usage: {home_disk.percent:.1f}%")

# Check current directory space
try:
    cwd_disk = psutil.disk_usage(str(Path.cwd()))
    print(f"\nCurrent directory:")
    print(f"  Total: {cwd_disk.total / 1e9:.2f} GB")
    print(f"  Used: {cwd_disk.used / 1e9:.2f} GB")
    print(f"  Free: {cwd_disk.free / 1e9:.2f} GB")
    print(f"  Usage: {cwd_disk.percent:.1f}%")
except:
    pass

# Warning if low on space
min_space_gb = 10  # Minimum recommended free space in GB
if disk.free / 1e9 < min_space_gb:
    print(f"\n⚠ WARNING: Low disk space! Only {disk.free / 1e9:.2f} GB free.")
    print("Consider:")
    print("  1. Using scratch/tmp directories (see next cell)")
    print("  2. Cleaning up old files")
    print("  3. Downloading only essential datasets")
else:
    print(f"\n✓ Sufficient disk space available ({disk.free / 1e9:.2f} GB free)")
print("="*60)

In [ ]:
import os
import sys
from pathlib import Path
import psutil

# Determine user-writable directory
# Try common supercomputer/user directories (prioritize scratch/tmp for more space)
user_home = Path.home()
possible_dirs = [
    Path("/scratch") / os.getenv("USER", "user"),  # Scratch usually has more space
    Path("/tmp") / os.getenv("USER", "user"),      # Temp often has more space
    user_home / "scratch",                         # User scratch
    user_home / "work",
    user_home / "tmp",
    Path.cwd(),  # Current working directory (last resort)
]

# Check disk space for each directory and prefer ones with more space
def get_dir_space(path):
    """Get free space for a directory"""
    try:
        return psutil.disk_usage(str(path)).free
    except:
        return 0

# Sort by available space (descending)
possible_dirs_with_space = [(d, get_dir_space(d)) for d in possible_dirs]
possible_dirs_with_space.sort(key=lambda x: x[1], reverse=True)
possible_dirs = [d for d, _ in possible_dirs_with_space]

# Show available directories with their space
print("="*60)
print("CHECKING AVAILABLE DIRECTORIES")
print("="*60)
for dir_path, space in possible_dirs_with_space:
    space_gb = space / 1e9
    try:
        # Test if writable
        test_file = dir_path / ".write_test"
        dir_path.mkdir(parents=True, exist_ok=True)
        test_file.write_text("test")
        test_file.unlink()
        writable = "✓ Writable"
    except:
        writable = "✗ Not writable"
    print(f"{str(dir_path):40s} {space_gb:8.2f} GB  {writable}")
print("="*60)

# Find first writable directory with sufficient space (prefer >10GB free)
WORK_DIR = None
min_space_gb = 10  # Minimum recommended space

for dir_path, space_gb in possible_dirs_with_space:
    space_gb_value = space_gb / 1e9
    try:
        dir_path.mkdir(parents=True, exist_ok=True)
        # Test write
        test_file = dir_path / ".write_test"
        test_file.write_text("test")
        test_file.unlink()
        
        # Prefer directories with more space
        if space_gb_value >= min_space_gb:
            WORK_DIR = dir_path / "D-en-ominators"
            print(f"\n✓ Selected directory with {space_gb_value:.2f} GB free: {dir_path}")
            break
    except (PermissionError, OSError):
        continue

# If no directory with sufficient space, use the one with most space
if WORK_DIR is None:
    for dir_path, space_gb in possible_dirs_with_space:
        space_gb_value = space_gb / 1e9
        try:
            dir_path.mkdir(parents=True, exist_ok=True)
            test_file = dir_path / ".write_test"
            test_file.write_text("test")
            test_file.unlink()
            WORK_DIR = dir_path / "D-en-ominators"
            print(f"\n⚠ Using directory with {space_gb_value:.2f} GB free (less than recommended {min_space_gb} GB): {dir_path}")
            print("   Consider cleaning up or using cleanup functions later.")
            break
        except (PermissionError, OSError):
            continue

if WORK_DIR is None:
    # Fallback to current directory
    WORK_DIR = Path.cwd() / "D-en-ominators"
    print(f"\n⚠ Using current directory as fallback: {WORK_DIR}")

# Set up directories
NEW_DIR = WORK_DIR / "new"
DATASETS_DIR = WORK_DIR / "datasets"
CHECKPOINTS_DIR = WORK_DIR / "checkpoints"

# Create directories
WORK_DIR.mkdir(parents=True, exist_ok=True)
NEW_DIR.mkdir(parents=True, exist_ok=True)
DATASETS_DIR.mkdir(parents=True, exist_ok=True)
CHECKPOINTS_DIR.mkdir(parents=True, exist_ok=True)

# Change to working directory
os.chdir(WORK_DIR)

# Add new folder to Python path
sys.path.insert(0, str(NEW_DIR))
sys.path.insert(0, str(WORK_DIR))

print("\n" + "="*60)
print("DIRECTORY SETUP")
print("="*60)
print(f"✓ Working directory: {WORK_DIR}")
print(f"✓ Code directory: {NEW_DIR}")
print(f"✓ Datasets directory: {DATASETS_DIR}")
print(f"✓ Checkpoints directory: {CHECKPOINTS_DIR}")
print(f"✓ Current working directory: {Path.cwd()}")

# Check space in selected directory
try:
    work_disk = psutil.disk_usage(str(WORK_DIR.parent))
    print(f"\n✓ Available space in {WORK_DIR.parent}: {work_disk.free / 1e9:.2f} GB")
    if work_disk.free / 1e9 < 5:
        print("  ⚠ WARNING: Low space! Consider using cleanup functions or downloading fewer datasets.")
except:
    pass
print("="*60)

In [ ]:
# Clone the repository from GitHub
import subprocess
import shutil

clone_dir = WORK_DIR.parent / "D-en-ominators_repo"
if clone_dir.exists():
    print(f"⚠ Repository directory exists, removing: {clone_dir}")
    shutil.rmtree(clone_dir)

print(f"Cloning repository to: {clone_dir}")
result = subprocess.run(
    ["git", "clone", "-b", "dev", "https://github.com/pragundamani/D-en-ominators.git", str(clone_dir)],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✓ Repository cloned successfully")
    
    # Copy code files from the 'new' folder
    source_new = clone_dir / "new"
    if source_new.exists():
        import glob
        for file in glob.glob(str(source_new / "*")):
            if os.path.isfile(file):
                shutil.copy2(file, NEW_DIR)
        print(f"✓ Copied code files from {source_new} to {NEW_DIR}")
    else:
        print(f"⚠ Warning: {source_new} not found after cloning")
else:
    print(f"✗ Error cloning repository: {result.stderr}")
    print("You may need to upload files manually (Option B)")

print(f"✓ Setup complete. Working directory: {WORK_DIR}")

In [ ]:
import zipfile
import shutil
from pathlib import Path

print("="*60)
print("MANUAL FILE UPLOAD")
print("="*60)
print(f"Upload directory: {NEW_DIR}")
print("\nInstructions:")
print("1. Upload a zip file containing all files from the 'new' folder")
print("2. Or upload individual Python files")
print("3. Files should be uploaded to the current working directory")
print(f"\nExpected upload location: {Path.cwd()}")

# Check for uploaded files in current directory
uploaded_files = list(Path.cwd().glob("*.zip")) + list(Path.cwd().glob("*.py"))

if uploaded_files:
    print(f"\n✓ Found {len(uploaded_files)} uploaded file(s):")
    for f in uploaded_files:
        print(f"  - {f.name}")
    
    # Process uploaded files
    for filename in uploaded_files:
        if filename.suffix == '.zip':
            print(f"\nExtracting {filename.name}...")
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall(NEW_DIR)
            print(f"✓ Extracted {filename.name} to {NEW_DIR}")
        elif filename.suffix == '.py':
            shutil.copy2(filename, NEW_DIR / filename.name)
            print(f"✓ Copied {filename.name} to {NEW_DIR}")
else:
    print("\n⚠ No uploaded files found in current directory")
    print("Please upload files to:", Path.cwd())

print("\n" + "="*60)
print("VERIFICATION")
print("="*60)
if NEW_DIR.exists():
    files_list = list(NEW_DIR.glob("*.py"))
    print(f"Found {len(files_list)} Python files in {NEW_DIR}")
    for f in files_list[:5]:
        print(f"  ✓ {f.name}")
    if len(files_list) > 5:
        print(f"  ... and {len(files_list) - 5} more files")
else:
    print(f"⚠ {NEW_DIR} does not exist yet")
print("="*60)

In [ ]:
import os
import sys
import subprocess
import requests
from pathlib import Path
from tqdm import tqdm
import shutil
import psutil

# Check disk space before starting
def check_disk_space(path, required_gb=5):
    """Check if there's enough disk space"""
    try:
        disk = psutil.disk_usage(str(path))
        free_gb = disk.free / 1e9
        if free_gb < required_gb:
            print(f"⚠ WARNING: Low disk space: {free_gb:.2f} GB free (need ~{required_gb} GB)")
            return False
        return True
    except:
        return True  # Assume OK if can't check

# Create datasets directory
DATASETS_DIR = Path("datasets")
DATASETS_DIR.mkdir(exist_ok=True)

# Check disk space
if not check_disk_space(DATASETS_DIR, required_gb=10):
    print("\n⚠ Consider cleaning up or using a different directory with more space")
    print("You can modify DATASETS_DIR to point to a scratch directory")

def run_command(cmd, check=True, show_progress=False):
    """Run a shell command and return the result"""
    try:
        if show_progress:
            process = subprocess.Popen(
                cmd, shell=True, stdout=subprocess.PIPE, 
                stderr=subprocess.STDOUT, text=True, bufsize=1
            )
            for line in process.stdout:
                print(f"  {line.strip()}")
            process.wait()
            if process.returncode != 0:
                print(f"  ✗ Command failed with exit code {process.returncode}")
                return False
            return True
        else:
            result = subprocess.run(cmd, shell=True, check=check, capture_output=True, text=True)
            if result.returncode != 0 and check:
                print(f"  ✗ Command failed: {result.stderr}")
                return False
            return result.returncode == 0
    except Exception as e:
        print(f"Error running command: {cmd}")
        print(f"Error: {e}")
        return False

def dataset_exists(output_dir):
    """Check if dataset already exists and has content"""
    output_path = DATASETS_DIR / output_dir
    if output_path.exists() and any(output_path.rglob("*")):
        # Check if it has meaningful content (more than just empty dirs)
        files = list(output_path.rglob("*"))
        non_empty = [f for f in files if f.is_file() and f.stat().st_size > 0]
        return len(non_empty) > 0
    return False

def download_huggingface_dataset(dataset_name, output_dir):
    """Download a Hugging Face dataset"""
    # Check if already exists
    if dataset_exists(output_dir):
        print(f"\n⏭️  Skipping {dataset_name} - already exists in {output_dir}")
        return True
    
    print(f"\n📦 Downloading Hugging Face dataset: {dataset_name}")
    try:
        from datasets import load_dataset
        output_path = DATASETS_DIR / output_dir
        output_path.mkdir(parents=True, exist_ok=True)
        
        with tqdm(desc="  Loading dataset", bar_format='{desc}: {elapsed}') as pbar:
            print(f"  Loading dataset {dataset_name}...")
            dataset = load_dataset(dataset_name)
            pbar.update(1)
        
        if isinstance(dataset, dict):
            splits = list(dataset.keys())
            with tqdm(total=len(splits), desc="  Saving splits", unit="split") as pbar:
                for split_name, split_data in dataset.items():
                    split_path = output_path / split_name
                    split_path.mkdir(exist_ok=True)
                    split_data.save_to_disk(str(split_path))
                    pbar.set_postfix({"split": split_name})
                    pbar.update(1)
                    print(f"  ✓ Saved {split_name} split")
        else:
            with tqdm(desc="  Saving dataset", bar_format='{desc}: {elapsed}') as pbar:
                dataset.save_to_disk(str(output_path))
                pbar.update(1)
                print(f"  ✓ Saved dataset")
        
        if not any(output_path.rglob("*")):
            print(f"  ✗ Warning: Dataset saved but no files found in {output_path}")
            return False
        
        print(f"  ✓ Verified files saved to {output_path}")
        return True
    except ImportError:
        print("  ⚠ Installing datasets library...")
        with tqdm(desc="  Installing", bar_format='{desc}: {elapsed}') as pbar:
            run_command(f"{sys.executable} -m pip install datasets", check=False)
            pbar.update(1)
        return download_huggingface_dataset(dataset_name, output_dir)
    except Exception as e:
        print(f"  ✗ Error: {e}")
        import traceback
        traceback.print_exc()
        return False

def download_github_repo(repo_url, output_dir, subpath=None):
    """Download a GitHub repository or specific subdirectory"""
    # Check if already exists
    if dataset_exists(output_dir):
        print(f"\n⏭️  Skipping {repo_url} - already exists in {output_dir}")
        return True
    
    print(f"\n📦 Downloading GitHub repository: {repo_url}")
    try:
        output_path = DATASETS_DIR / output_dir
        output_path.mkdir(parents=True, exist_ok=True)
        
        if "github.com" in repo_url:
            repo_path = repo_url.split("github.com/")[1].split("/tree/")[0]
            if "/tree/" in repo_url:
                branch = repo_url.split("/tree/")[1].split("/")[0]
                subpath = "/".join(repo_url.split("/tree/")[1].split("/")[1:])
            else:
                branch = "main"
            
            repo_url_clean = f"https://github.com/{repo_path}.git"
            
            print(f"  Cloning {repo_url_clean} (branch: {branch})...")
            with tqdm(desc="  Cloning repository", bar_format='{desc}: {elapsed}') as pbar:
                success = run_command(f"git clone --depth 1 --branch {branch} {repo_url_clean} {output_path}", 
                          check=False, show_progress=True)
                pbar.update(1)
            
            if not success:
                print(f"  ✗ Failed to clone repository")
                return False
            
            if not output_path.exists() or not any(output_path.iterdir()):
                print(f"  ✗ Clone completed but directory is empty")
                return False
            
            if subpath:
                subpath_full = output_path / subpath
                if subpath_full.exists():
                    print(f"  ✓ Found subpath: {subpath}")
                else:
                    print(f"  ⚠ Subpath {subpath} not found")
            
            print(f"  ✓ Successfully cloned to {output_path}")
            return True
        else:
            print(f"  ✗ Invalid GitHub URL: {repo_url}")
            return False
    except Exception as e:
        print(f"  ✗ Error: {e}")
        import traceback
        traceback.print_exc()
        return False

def download_kaggle_dataset(dataset_name, output_dir):
    """Download a Kaggle dataset"""
    # Check if already exists
    if dataset_exists(output_dir):
        print(f"\n⏭️  Skipping {dataset_name} - already exists in {output_dir}")
        return True
    
    print(f"\n📦 Downloading Kaggle dataset: {dataset_name}")
    try:
        output_path = DATASETS_DIR / output_dir
        output_path.mkdir(parents=True, exist_ok=True)
        
        try:
            import kaggle
        except ImportError:
            print("  ⚠ Installing kaggle library...")
            with tqdm(desc="  Installing", bar_format='{desc}: {elapsed}') as pbar:
                run_command(f"{sys.executable} -m pip install kaggle", check=False)
                pbar.update(1)
            import kaggle
        
        print(f"  Downloading {dataset_name}...")
        with tqdm(desc="  Downloading from Kaggle", bar_format='{desc}: {elapsed}') as pbar:
            kaggle.api.dataset_download_files(dataset_name, path=str(output_path), unzip=True)
            pbar.update(1)
        
        if not any(output_path.rglob("*")):
            print(f"  ✗ Warning: Download completed but no files found in {output_path}")
            return False
        
        print(f"  ✓ Downloaded to {output_path}")
        return True
    except Exception as e:
        print(f"  ✗ Error: {e}")
        import traceback
        traceback.print_exc()
        print(f"  ⚠ Note: Kaggle requires API credentials. Upload kaggle.json to /content/.kaggle/")
        return False

def download_file(url, output_path):
    """Download a file directly from URL"""
    output_file = DATASETS_DIR / output_path
    
    # Check if already exists
    if output_file.exists() and output_file.stat().st_size > 0:
        print(f"\n⏭️  Skipping {url} - file already exists: {output_file}")
        return True
    
    print(f"\n📦 Downloading file: {url}")
    try:
        output_file.parent.mkdir(parents=True, exist_ok=True)
        
        response = requests.get(url, stream=True, timeout=30)
        response.raise_for_status()
        
        total_size = int(response.headers.get('content-length', 0))
        
        with open(output_file, 'wb') as f:
            if total_size > 0:
                with tqdm(total=total_size, unit='B', unit_scale=True, unit_divisor=1024, 
                      desc="  Downloading", bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]') as pbar:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                            pbar.update(len(chunk))
            else:
                with tqdm(unit='B', unit_scale=True, unit_divisor=1024, 
                         desc="  Downloading", bar_format='{desc}: {n_fmt} [{elapsed}]') as pbar:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                            pbar.update(len(chunk))
        
        print(f"  ✓ Downloaded to {output_file}")
        return True
    except Exception as e:
        print(f"\n  ✗ Error: {e}")
        return False

# List of datasets to download with priority levels
# ESSENTIAL = Always download (needed for training)
# OPTIONAL = Download only if space allows

datasets = [
    {
        "name": "Homogenized Finalized Dataset",
        "type": "github",
        "url": "https://github.com/pragundamani/D-en-ominators/tree/dev/finalized_dataset/homogenized",
        "output": "homogenized_dataset",
        "subpath": "finalized_dataset/homogenized",
        "estimated_size_gb": 3
    }
] dataset['output'])
        elif dataset['type'] == 'github':
            success = download_github_repo(dataset['url'], dataset['output'], dataset.get('subpath'))
        elif dataset['type'] == 'kaggle':
            success = download_kaggle_dataset(dataset['dataset_id'], dataset['output'])
        elif dataset['type'] == 'direct':
            success = download_file(dataset['url'], dataset['output'])
        
        results.append({
            "name": dataset['name'],
            "success": success
        })
        
        overall_pbar.update(1)

# Summary
print("\n" + "=" * 80)
print("DOWNLOAD SUMMARY")
print("=" * 80)
for result in results:
    status = "✓" if result['success'] else "✗"
    print(f"{status} {result['name']}")

successful = sum(1 for r in results if r['success'])
# Create a mapping of dataset names to output directories
dataset_map = {d['name']: d['output'] for d in datasets}
skipped = sum(1 for r in results if r['success'] and dataset_exists(dataset_map.get(r['name'], '')))
print(f"\nSuccessfully processed: {successful}/{len(results)} datasets")
if skipped > 0:
    print(f"Skipped (already existed): {skipped} datasets")
print(f"Datasets saved to: {DATASETS_DIR.absolute()}")

# Check disk space after downloads
print("\n" + "="*60)
print("DISK SPACE AFTER DOWNLOADS")
print("="*60)
try:
    disk = psutil.disk_usage(str(DATASETS_DIR))
    print(f"Free space: {disk.free / 1e9:.2f} GB")
    if disk.free / 1e9 < 5:
        print("⚠ WARNING: Low disk space remaining!")
except:
    pass
print("="*60)


In [ ]:
# Cleanup functions to free disk space

def cleanup_old_checkpoints(keep_best=True, keep_latest=3):
    """Remove old checkpoints, keeping only best and latest N"""
    if not CHECKPOINTS_DIR.exists():
        return 0
    
    checkpoints = list(CHECKPOINTS_DIR.glob("*.pth"))
    if len(checkpoints) <= keep_latest + (1 if keep_best else 0):
        print("No old checkpoints to remove")
        return 0
    
    # Keep best model
    best_model = CHECKPOINTS_DIR / "best_model.pth"
    kept = [best_model] if keep_best and best_model.exists() else []
    
    # Sort by modification time, keep latest N
    checkpoints.sort(key=lambda x: x.stat().st_mtime, reverse=True)
    kept.extend(checkpoints[:keep_latest])
    
    # Remove others
    removed_size = 0
    for cp in checkpoints:
        if cp not in kept:
            size = cp.stat().st_size
            cp.unlink()
            removed_size += size
            print(f"  Removed: {cp.name} ({size / 1e6:.2f} MB)")
    
    print(f"✓ Freed {removed_size / 1e9:.2f} GB from old checkpoints")
    return removed_size

def cleanup_temporary_files():
    """Remove temporary files and caches"""
    removed_size = 0
    
    # Remove __pycache__
    for pycache in WORK_DIR.rglob("__pycache__"):
        if pycache.is_dir():
            size = sum(f.stat().st_size for f in pycache.rglob("*") if f.is_file())
            shutil.rmtree(pycache)
            removed_size += size
            print(f"  Removed: {pycache}")
    
    # Remove .pyc files
    for pyc in WORK_DIR.rglob("*.pyc"):
        size = pyc.stat().st_size
        pyc.unlink()
        removed_size += size
    
    # Remove .DS_Store files
    for ds_store in WORK_DIR.rglob(".DS_Store"):
        size = ds_store.stat().st_size
        ds_store.unlink()
        removed_size += size
    
    print(f"✓ Freed {removed_size / 1e6:.2f} MB from temporary files")
    return removed_size

def cleanup_dataset_cache():
    """Remove Hugging Face cache if needed"""
    hf_cache = Path.home() / ".cache" / "huggingface"
    if hf_cache.exists():
        size = sum(f.stat().st_size for f in hf_cache.rglob("*") if f.is_file())
        print(f"⚠ Hugging Face cache found: {size / 1e9:.2f} GB")
        response = input("Remove Hugging Face cache? (y/n): ").lower()
        if response == 'y':
            shutil.rmtree(hf_cache)
            print(f"✓ Freed {size / 1e9:.2f} GB")
            return size
    return 0

def show_disk_usage():
    """Show disk usage by directory"""
    print("="*60)
    print("DISK USAGE BREAKDOWN")
    print("="*60)
    
    dirs_to_check = {
        "Datasets": DATASETS_DIR,
        "Checkpoints": CHECKPOINTS_DIR,
        "Code": NEW_DIR,
        "Working Directory": WORK_DIR,
    }
    
    total_size = 0
    for name, path in dirs_to_check.items():
        if path.exists():
            size = sum(f.stat().st_size for f in path.rglob("*") if f.is_file())
            total_size += size
            print(f"{name:20s}: {size / 1e9:8.2f} GB")
    
    print(f"{'Total':20s}: {total_size / 1e9:8.2f} GB")
    print("="*60)

# Run cleanup
print("="*60)
print("CLEANUP OPTIONS")
print("="*60)
print("Uncomment the cleanup functions you want to run:\n")

# Uncomment to run cleanups:
# cleanup_old_checkpoints(keep_best=True, keep_latest=3)
# cleanup_temporary_files()
# cleanup_dataset_cache()  # Interactive - asks for confirmation
# show_disk_usage()

print("\nTo free up space, uncomment the cleanup functions above and run this cell.")
print("="*60)


In [ ]:
from pathlib import Path

print("Checking setup...")
print("\n1. Code files:")
required_files = ['hieroglyph_dataset.py', 'model.py', 'train.py', 'inference.py']
for f in required_files:
    file_path = NEW_DIR / f
    status = "✓" if file_path.exists() else "✗"
    print(f"  {status} {f}")

print("\n2. Datasets:")
if DATASETS_DIR.exists():
    dataset_dirs = [d for d in DATASETS_DIR.iterdir() if d.is_dir()]
    print(f"  ✓ Found {len(dataset_dirs)} dataset directories")
    for d in dataset_dirs[:5]:
        print(f"    - {d.name}")
else:
    print("  ✗ Datasets directory not found")

print("\n3. Python path:")
print(f"  ✓ NEW_DIR in path: {str(NEW_DIR) in sys.path}")

print("\n4. GPU:")
import torch
print(f"  {'✓' if torch.cuda.is_available() else '✗'} CUDA available: {torch.cuda.is_available()}")

In [ ]:
# Preload dataset into RAM for faster training
import json
import numpy as np
from PIL import Image
from pathlib import Path
import torch
from tqdm import tqdm

# Find the dataset paths
json_path = None
images_dir = None

for json_file in DATASETS_DIR.rglob("gardiner_hieroglyphs_with_unicode_hex.json"):
    json_path = str(json_file)
    utf_pngs = json_file.parent / "utf-pngs"
    if utf_pngs.exists():
        images_dir = str(utf_pngs)
    break

# Check if we should use RAM caching
USE_RAM_CACHE = True  # Set to False to disable RAM caching
RAM_CACHE_FILE = WORK_DIR / "dataset_ram_cache.pt"

def estimate_dataset_size(images_dir):
    """Estimate dataset size in memory"""
    image_files = list(Path(images_dir).glob("*.png"))
    if not image_files:
        return 0
    
    # Sample a few images to estimate
    sample_size = min(10, len(image_files))
    total_size = 0
    for img_file in image_files[:sample_size]:
        try:
            img = Image.open(img_file)
            # Estimate: width * height * 3 channels * 4 bytes (float32) * 2 (augmentation overhead)
            size = img.size[0] * img.size[1] * 3 * 4 * 2
            total_size += size
        except:
            pass
    
    avg_size = (total_size / sample_size) if sample_size > 0 else 0
    estimated_gb = (avg_size * len(image_files)) / 1e9
    return estimated_gb

def preload_dataset_to_ram(json_path, images_dir, cache_file=None):
    """Preload all images into RAM"""
    print("="*60)
    print("PRELOADING DATASET INTO RAM")
    print("="*60)
    
    # Check if cache exists
    if cache_file and cache_file.exists():
        print(f"✓ Found existing RAM cache: {cache_file}")
        response = input("Load from cache? (y/n, default=y): ").lower().strip()
        if response != 'n':
            print("Loading dataset from RAM cache...")
            cache_data = torch.load(cache_file, map_location='cpu')
            print(f"✓ Loaded {len(cache_data['images'])} images from cache")
            print(f"  Memory usage: ~{cache_data.get('memory_gb', 0):.2f} GB")
            return cache_data
    
    # Load JSON metadata
    with open(json_path, 'r', encoding='utf-8') as f:
        hieroglyph_data = json.load(f)
    
    # Estimate memory needed
    estimated_gb = estimate_dataset_size(images_dir)
    print(f"Estimated dataset size: ~{estimated_gb:.2f} GB in RAM")
    
    # Check available RAM and GPU VRAM (optimize for 64GB RAM + 48GB VRAM RTX 8000)
    mem = psutil.virtual_memory()
    total_ram_gb = mem.total / 1e9
    available_ram_gb = mem.available / 1e9
    print(f"Total RAM: {total_ram_gb:.2f} GB")
    print(f"Available RAM: {available_ram_gb:.2f} GB")
    
    # Check GPU VRAM if available
    gpu_vram_gb = 0
    if torch.cuda.is_available():
        gpu_props = torch.cuda.get_device_properties(0)
        gpu_vram_gb = gpu_props.total_memory / 1e9
        print(f"GPU VRAM: {gpu_vram_gb:.2f} GB (RTX 8000 has 48GB)")
        print(f"  → GPU VRAM will be used for model/batches during training")
        print(f"  → RAM will be used for dataset storage (faster than disk)")
    
    # Use up to 90% of available RAM for 64GB systems (more aggressive for large RAM)
    max_ram_usage = available_ram_gb * 0.9 if total_ram_gb >= 60 else available_ram_gb * 0.8
    
    if estimated_gb > max_ram_usage:
        if total_ram_gb >= 60:
            print(f"⚠ Dataset ({estimated_gb:.2f} GB) exceeds safe limit ({max_ram_usage:.2f} GB)")
            print("  ✓ 64GB RAM detected - proceeding with preload (will use available RAM)")
        else:
            print(f"⚠ WARNING: Dataset may exceed available RAM!")
            response = input("Continue anyway? (y/n): ").lower().strip()
            if response != 'y':
                print("Skipping RAM preload. Will use disk-based loading.")
                return None
    
    # Load all images into memory (optimized for 64GB RAM with parallel loading)
    images_dir_path = Path(images_dir)
    image_files = list(images_dir_path.glob("*.png"))
    
    print(f"\nLoading {len(image_files)} images into RAM (using 64GB RAM)...")
    print("  Using parallel loading for faster processing...")
    
    images_dict = {}
    metadata_dict = {}
    failed = 0
    
    # Use multiprocessing for faster loading on systems with many cores
    from concurrent.futures import ThreadPoolExecutor, as_completed
    
    def load_image(img_file):
        """Load a single image"""
        try:
            gardiner_num = img_file.stem
            img = Image.open(img_file).convert('RGB')
            img_array = np.array(img, dtype=np.uint8)
            
            # Get metadata
            metadata = None
            if gardiner_num in hieroglyph_data:
                metadata = hieroglyph_data[gardiner_num]
            
            return gardiner_num, img_array, metadata, None
        except Exception as e:
            return None, None, None, str(e)
    
    # Load images with parallel processing (faster on multi-core systems)
    max_workers = min(8, os.cpu_count() or 4)  # Use up to 8 workers
    
    with tqdm(total=len(image_files), desc="Loading images", unit="img") as pbar:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = {executor.submit(load_image, img_file): img_file for img_file in image_files}
            
            for future in as_completed(futures):
                gardiner_num, img_array, metadata, error = future.result()
                if gardiner_num and img_array is not None:
                    images_dict[gardiner_num] = img_array
                    if metadata:
                        metadata_dict[gardiner_num] = metadata
                else:
                    failed += 1
                pbar.update(1)
    
    # Calculate actual memory usage
    total_memory = sum(img.nbytes for img in images_dict.values())
    memory_gb = total_memory / 1e9
    
    cache_data = {
        'images': images_dict,
        'metadata': metadata_dict,
        'json_path': json_path,
        'images_dir': images_dir,
        'memory_gb': memory_gb,
        'num_images': len(images_dict)
    }
    
    # Save cache for future use
    if cache_file:
        print(f"\nSaving RAM cache to {cache_file}...")
        torch.save(cache_data, cache_file)
        print("✓ Cache saved")
    
    print(f"\n✓ Preloaded {len(images_dict)} images into RAM")
    print(f"  Memory usage: ~{memory_gb:.2f} GB")
    print(f"  Failed to load: {failed} images")
    
    # Show memory utilization
    mem_after = psutil.virtual_memory()
    print(f"\nMemory Status:")
    print(f"  RAM used by dataset: {memory_gb:.2f} GB")
    print(f"  Total RAM used: {(mem_after.used / 1e9):.2f} GB / {total_ram_gb:.2f} GB")
    print(f"  Available RAM remaining: {(mem_after.available / 1e9):.2f} GB")
    
    if torch.cuda.is_available() and gpu_vram_gb > 0:
        print(f"  GPU VRAM available: {gpu_vram_gb:.2f} GB (for model/batch data)")
        print(f"  ✓ Training will use GPU VRAM for model, RAM for data (optimal setup)")
    
    print("="*60)
    
    return cache_data

# Preload if enabled and dataset found
ram_cache_data = None
if USE_RAM_CACHE and json_path and images_dir:
    ram_cache_data = preload_dataset_to_ram(json_path, images_dir, RAM_CACHE_FILE)
    if ram_cache_data:
        print("\n✓ Dataset ready in RAM! Training will be much faster.")
    else:
        print("\n⚠ RAM preload skipped or failed. Will use disk-based loading.")
elif not json_path or not images_dir:
    print("⚠ Dataset not found. Skipping RAM preload.")
else:
    print("⚠ RAM caching disabled. Set USE_RAM_CACHE = True to enable.")


In [ ]:
# Use dataset paths (already found in previous cell if RAM cache was loaded)
# If not found, search again
if 'json_path' not in locals() or not json_path:
    json_path = None
    images_dir = None
    
    # Search for the JSON file
    for json_file in DATASETS_DIR.rglob("gardiner_hieroglyphs_with_unicode_hex.json"):
        json_path = str(json_file)
        # Look for utf-pngs in the same parent directory
        utf_pngs = json_file.parent / "utf-pngs"
        if utf_pngs.exists():
            images_dir = str(utf_pngs)
        break

if json_path and images_dir:
    print(f"✓ Found JSON: {json_path}")
    print(f"✓ Found images: {images_dir}")
    print(f"✓ Checkpoints will be saved to: {CHECKPOINTS_DIR}")
    
    # Check if RAM cache is available
    use_ram = ram_cache_data is not None
    if use_ram:
        print(f"✓ Using RAM cache: {ram_cache_data['num_images']} images (~{ram_cache_data.get('memory_gb', 0):.2f} GB)")
        print("  Training will be faster with in-memory data!")
    else:
        print("  Using disk-based loading (slower but uses less RAM)")
    
    print("\nStarting LoRA training...")
    print("="*60)
    print("LoRA (Low-Rank Adaptation) Training")
    print("  - Only trains a small number of parameters (~1-5% of model)")
    print("  - Much faster training and lower memory usage")
    print("  - Maintains model performance")
    print("="*60)
    
    # Set environment variables for GPU
    os.environ["CUDA_VISIBLE_DEVICES"] = "0" if torch.cuda.is_available() else ""
    
    # RTX 8000 optimizations - can handle larger batch sizes with 48GB VRAM
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        if "RTX 8000" in gpu_name or "8000" in gpu_name:
            batch_size = "64"  # RTX 8000 can handle larger batches
            print("✓ RTX 8000 detected - Using batch size 64 (48GB VRAM)")
        else:
            batch_size = "32"  # Default for other GPUs
    else:
        batch_size = "16"  # Smaller for CPU
    
    # Write train_lora.py to NEW_DIR (embedded in notebook for HPC)
    train_lora_path = NEW_DIR / "train_lora.py"
    if not train_lora_path.exists():
        print(f"Creating train_lora.py in {NEW_DIR}...")
        
        train_lora_code = '''"""
LoRA Training Script for Hieroglyph Classifier
Uses PEFT (Parameter-Efficient Fine-Tuning) with LoRA
"""

import os
import json
import argparse
from pathlib import Path
from datetime import datetime
from typing import Dict, Optional

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from PIL import Image
from tqdm import tqdm
import timm
from peft import LoraConfig, get_peft_model, TaskType

from hieroglyph_dataset import HieroglyphDataset, get_transforms
from model import create_model


class LoRATrainer:
    """Training manager with LoRA for hieroglyph classifier."""
    
    def __init__(
        self,
        model: nn.Module,
        train_loader: DataLoader,
        val_loader: DataLoader,
        device: str = "cuda",
        learning_rate: float = 1e-4,
        weight_decay: float = 1e-4,
        class_weights: Optional[torch.Tensor] = None,
        use_focal_loss: bool = True,
        focal_gamma: float = 2.0,
        label_smoothing: float = 0.1
    ):
        # Apply LoRA to the model
        print("\\n" + "="*60)
        print("APPLYING LoRA TO MODEL")
        print("="*60)
        
        # LoRA configuration
        lora_config = LoraConfig(
            task_type=TaskType.FEATURE_EXTRACTION,  # For vision models
            r=16,  # Rank (low-rank dimension)
            lora_alpha=32,  # LoRA alpha scaling
            target_modules=["qkv", "proj", "fc1", "fc2"],  # Target attention and MLP layers
            lora_dropout=0.1,
            bias="none",
        )
        
        # Convert model to PEFT with LoRA
        self.model = get_peft_model(model, lora_config)
        
        # Print trainable parameters
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in self.model.parameters())
        print(f"Total parameters: {total_params:,}")
        print(f"Trainable parameters (LoRA): {trainable_params:,}")
        print(f"Trainable %: {100 * trainable_params / total_params:.2f}%")
        print("="*60)
        
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.class_weights = class_weights
        
        # Loss function
        if use_focal_loss:
            self.criterion = FocalLoss(alpha=class_weights, gamma=focal_gamma)
            print(f"Using Focal Loss (gamma={focal_gamma})")
        else:
            if class_weights is not None:
                self.criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=label_smoothing)
            else:
                self.criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
        
        # Optimizer - only optimize LoRA parameters
        self.optimizer = optim.AdamW(
            self.model.parameters(),  # Only LoRA parameters are trainable
            lr=learning_rate,
            weight_decay=weight_decay
        )
        
        # Learning rate scheduler
        self.scheduler = ReduceLROnPlateau(
            self.optimizer,
            mode='min',
            factor=0.5,
            patience=3
        )
        
        # Training history
        self.history = {
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': []
        }
        
        self.best_val_acc = 0.0
        self.best_model_state = None
    
    def train_epoch(self) -> Dict[str, float]:
        """Train for one epoch."""
        self.model.train()
        running_loss = 0.0
        all_preds = []
        all_labels = []
        
        pbar = tqdm(self.train_loader, desc="Training")
        for images, labels, _ in pbar:
            images = images.to(self.device)
            labels = labels.to(self.device)
            
            # Forward pass
            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.criterion(outputs, labels)
            
            # Backward pass
            loss.backward()
            self.optimizer.step()
            
            # Statistics
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            # Update progress bar
            pbar.set_postfix({'loss': loss.item()})
        
        epoch_loss = running_loss / len(self.train_loader)
        epoch_acc = accuracy_score(all_labels, all_preds)
        
        return {'loss': epoch_loss, 'acc': epoch_acc}
    
    def validate(self) -> Dict[str, float]:
        """Validate the model."""
        self.model.eval()
        running_loss = 0.0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            pbar = tqdm(self.val_loader, desc="Validation")
            for images, labels, _ in pbar:
                images = images.to(self.device)
                labels = labels.to(self.device)
                
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                
                running_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                
                pbar.set_postfix({'loss': loss.item()})
        
        epoch_loss = running_loss / len(self.val_loader)
        epoch_acc = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds, average='weighted')
        
        return {'loss': epoch_loss, 'acc': epoch_acc, 'f1': f1}
    
    def train(self, num_epochs: int, save_dir: Path) -> Dict:
        """Train the model."""
        save_dir.mkdir(parents=True, exist_ok=True)
        
        print(f"\\n{'='*60}")
        print(f"TRAINING WITH LoRA")
        print(f"{'='*60}")
        print(f"Epochs: {num_epochs}")
        print(f"Save directory: {save_dir}")
        print(f"{'='*60}\\n")
        
        for epoch in range(1, num_epochs + 1):
            print(f"Epoch {epoch}/{num_epochs}")
            print("-" * 60)
            
            # Train
            train_metrics = self.train_epoch()
            self.history['train_loss'].append(train_metrics['loss'])
            self.history['train_acc'].append(train_metrics['acc'])
            
            # Validate
            val_metrics = self.validate()
            self.history['val_loss'].append(val_metrics['loss'])
            self.history['val_acc'].append(val_metrics['acc'])
            
            # Update learning rate
            self.scheduler.step(val_metrics['loss'])
            
            # Print metrics
            print(f"Train Loss: {train_metrics['loss']:.4f}, Train Acc: {train_metrics['acc']:.4f}")
            print(f"Val Loss: {val_metrics['loss']:.4f}, Val Acc: {val_metrics['acc']:.4f}, Val F1: {val_metrics['f1']:.4f}")
            print(f"LR: {self.optimizer.param_groups[0]['lr']:.6f}")
            
            # Save best model
            if val_metrics['acc'] > self.best_val_acc:
                self.best_val_acc = val_metrics['acc']
                self.best_model_state = self.model.state_dict()
                
                # Save LoRA adapters
                best_path = save_dir / "best_model_lora.pth"
                self.model.save_pretrained(str(save_dir / "lora_adapters"))
                torch.save({
                    'model_state_dict': self.best_model_state,
                    'epoch': epoch,
                    'val_acc': self.best_val_acc,
                    'history': self.history
                }, best_path)
                print(f"✓ Saved best model (val_acc: {self.best_val_acc:.4f})")
            
            print()
        
        return self.history


class FocalLoss(nn.Module):
    """Focal Loss for handling imbalanced classes."""
    def __init__(self, alpha=None, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    
    def forward(self, inputs, targets):
        ce_loss = nn.functional.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        return focal_loss.mean()


def main():
    parser = argparse.ArgumentParser(description='Train hieroglyph classifier with LoRA')
    parser.add_argument('--json_path', type=str, required=True, help='Path to JSON metadata file')
    parser.add_argument('--images_dir', type=str, required=True, help='Path to images directory')
    parser.add_argument('--ram_cache', type=str, default=None, help='Path to RAM cache file')
    parser.add_argument('--model_name', type=str, default='vit_base_patch16_224', help='Timm model name')
    parser.add_argument('--batch_size', type=int, default=32, help='Batch size')
    parser.add_argument('--num_epochs', type=int, default=50, help='Number of epochs')
    parser.add_argument('--learning_rate', type=float, default=1e-4, help='Learning rate')
    parser.add_argument('--weight_decay', type=float, default=1e-4, help='Weight decay')
    parser.add_argument('--val_split', type=float, default=0.2, help='Validation split ratio')
    parser.add_argument('--save_dir', type=str, required=True, help='Directory to save checkpoints')
    parser.add_argument('--use_focal_loss', action='store_true', help='Use focal loss')
    parser.add_argument('--focal_gamma', type=float, default=2.0, help='Focal loss gamma')
    parser.add_argument('--label_smoothing', type=float, default=0.1, help='Label smoothing')
    
    args = parser.parse_args()
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    
    # Load dataset
    print("\\nLoading dataset...")
    if args.ram_cache and Path(args.ram_cache).exists():
        print(f"Loading from RAM cache: {args.ram_cache}")
        cache_data = torch.load(args.ram_cache, map_location='cpu')
        # Use regular dataset but we'll need to modify it to use RAM cache
        # For now, use regular dataset loading
        full_dataset = HieroglyphDataset(
            json_path=cache_data.get('json_path', args.json_path),
            images_dir=cache_data.get('images_dir', args.images_dir),
            transform=get_transforms(train=True)
        )
        print("Note: RAM cache loaded but dataset will load from disk. To use RAM cache, modify HieroglyphDataset.__getitem__")
    else:
        full_dataset = HieroglyphDataset(
            json_path=args.json_path,
            images_dir=args.images_dir,
            transform=get_transforms(train=True)
        )
    
    # Split dataset
    train_size = int((1 - args.val_split) * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        full_dataset, [train_size, val_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    # Compute class weights
    train_labels = [full_dataset[i][1] for i in train_dataset.indices]
    unique_labels = np.unique(train_labels)
    class_weights = compute_class_weight('balanced', classes=unique_labels, y=train_labels)
    label_to_weight = {label: weight for label, weight in zip(unique_labels, class_weights)}
    class_weights = torch.FloatTensor(class_weights).to(device)
    
    # Create weighted sampler
    sample_weights = np.array([label_to_weight.get(label, 1.0) for label in train_labels])
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        sampler=sampler,
        num_workers=0,
        pin_memory=False
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=False
    )
    
    # Create model
    print("\\nCreating model...")
    model = create_model(
        model_name=args.model_name,
        num_classes=full_dataset.num_classes,
        pretrained=True,
        dropout_rate=0.3,
        device=device
    )
    
    # Create LoRA trainer
    trainer = LoRATrainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        device=device,
        learning_rate=args.learning_rate,
        weight_decay=args.weight_decay,
        class_weights=class_weights,
        use_focal_loss=args.use_focal_loss,
        focal_gamma=args.focal_gamma,
        label_smoothing=args.label_smoothing if not args.use_focal_loss else 0.0
    )
    
    # Train
    save_dir = Path(args.save_dir)
    history = trainer.train(
        num_epochs=args.num_epochs,
        save_dir=save_dir
    )
    
    # Save training history
    history_path = save_dir / "training_history.json"
    with open(history_path, 'w') as f:
        json.dump(history, f, indent=2)
    print(f"\\nTraining history saved to {history_path}")
    print("\\nTraining completed!")


if __name__ == "__main__":
    main()
'''
        
        with open(train_lora_path, 'w') as f:
            f.write(train_lora_code)
        print("✓ Created train_lora.py (embedded in notebook)")
    
    # Create a wrapper script that uses RAM cache if available
    if use_ram:
        # Save RAM cache path for training script
        ram_cache_env = str(RAM_CACHE_FILE)
        os.environ["RAM_CACHE_PATH"] = ram_cache_env
        print(f"✓ RAM cache path set: {ram_cache_env}")
        
        train_cmd = [
            sys.executable,
            str(train_lora_path),
            "--ram_cache", ram_cache_env,
            "--model_name", "vit_base_patch16_224",
            "--batch_size", batch_size,
            "--num_epochs", "50",
            "--learning_rate", "1e-4",
            "--save_dir", str(CHECKPOINTS_DIR),
            "--val_split", "0.2",
            "--use_focal_loss"  # Use focal loss for imbalanced classes
        ]
    else:
        train_cmd = [
            sys.executable,
            str(train_lora_path),
            "--json_path", json_path,
            "--images_dir", images_dir,
            "--model_name", "vit_base_patch16_224",
            "--batch_size", batch_size,
            "--num_epochs", "50",
            "--learning_rate", "1e-4",
            "--save_dir", str(CHECKPOINTS_DIR),
            "--val_split", "0.2",
            "--use_focal_loss"  # Use focal loss for imbalanced classes
        ]
    
    print(f"\nRunning LoRA training: {' '.join(train_cmd)}")
    result = subprocess.run(train_cmd, cwd=str(NEW_DIR))
    
    if result.returncode == 0:
        print("\n✓ LoRA training completed successfully!")
        print(f"✓ LoRA adapters saved to: {CHECKPOINTS_DIR / 'lora_adapters'}")
        print(f"✓ Best model saved to: {CHECKPOINTS_DIR / 'best_model_lora.pth'}")
    else:
        print(f"\n✗ LoRA training failed with exit code {result.returncode}")
else:
    print("✗ Could not find dataset files. Please check the dataset paths.")
    print(f"\nSearched in: {DATASETS_DIR}")
    print("\nAvailable directories:")
    for d in DATASETS_DIR.iterdir():
        if d.is_dir():
            print(f"  - {d}")

In [ ]:
# Set paths (update these based on your setup)
checkpoint_path = CHECKPOINTS_DIR / "best_model.pth"

print("="*60)
print("INFERENCE SETUP")
print("="*60)
print(f"Checkpoint path: {checkpoint_path}")
print(f"JSON path: {json_path if json_path else 'Not found'}")
print(f"Images directory: {images_dir if images_dir else 'Not found'}")

if json_path and images_dir and checkpoint_path.exists():
    print("\n✓ All required files found!")
    print("\nTo run inference:")
    print("1. Upload an image file to the current directory")
    print(f"2. Update the image_path variable below")
    print("3. Run the inference command")
    
    # Example: Find uploaded images in current directory
    uploaded_images = list(Path.cwd().glob("*.png")) + list(Path.cwd().glob("*.jpg")) + list(Path.cwd().glob("*.jpeg"))
    
    if uploaded_images:
        image_path = str(uploaded_images[0])
        print(f"\n✓ Found uploaded image: {image_path}")
        print("\nRunning inference...")
        
        import subprocess
        inference_cmd = [
            sys.executable,
            str(NEW_DIR / "inference.py"),
            "--checkpoint", str(checkpoint_path),
            "--json_path", json_path,
            "--images_dir", images_dir,
            "--image", image_path,
            "--top_k", "5"
        ]
        
        result = subprocess.run(inference_cmd, cwd=str(NEW_DIR))
        if result.returncode == 0:
            print("\n✓ Inference completed successfully!")
        else:
            print(f"\n✗ Inference failed with exit code {result.returncode}")
    else:
        print("\n⚠ No image files found in current directory")
        print("Please upload an image file (.png, .jpg, .jpeg) to:", Path.cwd())
        print("\nExample inference command (after uploading image):")
        print(f"python {NEW_DIR / 'inference.py'} \\")
        print(f"    --checkpoint {checkpoint_path} \\")
        print(f"    --json_path {json_path} \\")
        print(f"    --images_dir {images_dir} \\")
        print(f"    --image <your_image_file> \\")
        print(f"    --top_k 5")
else:
    missing = []
    if not json_path:
        missing.append("JSON file")
    if not images_dir:
        missing.append("Images directory")
    if not checkpoint_path.exists():
        missing.append("Checkpoint file")
    print(f"\n✗ Missing required files: {', '.join(missing)}")
    print("Please train the model first or check paths.")
print("="*60)

In [ ]:
checkpoint_path = CHECKPOINTS_DIR / "best_model.pth"

if json_path and images_dir and checkpoint_path.exists():
    print("="*60)
    print("EVALUATION")
    print("="*60)
    print(f"Checkpoint: {checkpoint_path}")
    print(f"JSON: {json_path}")
    print(f"Images: {images_dir}")
    print("\nRunning evaluation...")
    
    import subprocess
    eval_cmd = [
        sys.executable,
        str(NEW_DIR / "evaluate.py"),
        "--checkpoint", str(checkpoint_path),
        "--json_path", json_path,
        "--images_dir", images_dir,
        "--output", str(WORK_DIR / "evaluation_results.json")
    ]
    
    result = subprocess.run(eval_cmd, cwd=str(NEW_DIR))
    
    if result.returncode == 0:
        print(f"\n✓ Evaluation completed!")
        print(f"Results saved to: {WORK_DIR / 'evaluation_results.json'}")
    else:
        print(f"\n✗ Evaluation failed with exit code {result.returncode}")
    print("="*60)
else:
    missing = []
    if not json_path:
        missing.append("JSON file")
    if not images_dir:
        missing.append("Images directory")
    if not checkpoint_path.exists():
        missing.append("Checkpoint file")
    print(f"✗ Missing required files: {', '.join(missing)}")
    print("Please train the model first.")

In [ ]:
import shutil
from pathlib import Path

print("="*60)
print("SAVE RESULTS")
print("="*60)

# List what to save
save_items = {
    "Checkpoints": CHECKPOINTS_DIR,
    "Evaluation Results": WORK_DIR / "evaluation_results.json",
    "Training History": CHECKPOINTS_DIR / "training_history.json",
}

print("\nAvailable items to save:")
for name, path in save_items.items():
    if path.exists():
        if path.is_dir():
            size = sum(f.stat().st_size for f in path.rglob('*') if f.is_file()) / 1e6
            print(f"  ✓ {name}: {path} ({size:.2f} MB)")
        else:
            size = path.stat().st_size / 1e6
            print(f"  ✓ {name}: {path} ({size:.2f} MB)")
    else:
        print(f"  ✗ {name}: {path} (not found)")

# Save to user home or specified location
save_location = Path.home() / "hieroglyph_classifier_results"
save_location.mkdir(parents=True, exist_ok=True)

print(f"\nSaving to: {save_location}")

for name, source in save_items.items():
    if source.exists():
        if source.is_dir():
            dest = save_location / source.name
            if dest.exists():
                shutil.rmtree(dest)
            shutil.copytree(source, dest)
            print(f"  ✓ Saved {name} to {dest}")
        else:
            dest = save_location / source.name
            shutil.copy2(source, dest)
            print(f"  ✓ Saved {name} to {dest}")

print(f"\n✓ All results saved to: {save_location}")
print("="*60)

In [ ]:
# ============================================================================
# STEP-BY-STEP INSTRUCTIONS
# ============================================================================
print("="*80)
print("STEP-BY-STEP INSTRUCTIONS")
print("="*80)
print("")
print("CELL 1: Install Dependencies")
print("  - Checks and installs required packages")
print("  - Detects GPU (RTX 8000) and optimizes settings")
print("  - Shows system info (RAM, disk space)")
print("")
print("CELL 2: Set Up Directories")
print("  - Finds writable directory (scratch/tmp preferred)")
print("  - Creates working directories")
print("  - Sets up Python paths")
print("")
print("CELL 3: Clone Repository OR Upload Files")
print("  Option A: Clones from GitHub (recommended)")
print("  Option B: Processes uploaded files from current directory")
print("")
print("CELL 4: Download Datasets")
print("  - Automatically filters datasets based on available disk space")
print("  - Always downloads essential dataset (Homogenized)")
print("  - Downloads optional datasets only if space allows")
print("  - Skips datasets that already exist")
print("")
print("CELL 5: Preload Dataset into RAM")
print("  - Loads all training images into RAM (uses 64GB RAM)")
print("  - Creates cache file for future use")
print("  - Eliminates disk I/O during training")
print("  - Set USE_RAM_CACHE = True to enable")
print("")
print("CELL 6: Verify Setup")
print("  - Checks that all required files are present")
print("  - Verifies dataset paths")
print("  - Confirms GPU availability")
print("")
print("CELL 7: Run Training")
print("  - Uses RAM cache if available (much faster)")
print("  - Automatically uses batch size 64 for RTX 8000")
print("  - Saves checkpoints during training")
print("")
print("CELL 8: Run Inference (Optional)")
print("  - Classifies hieroglyph images")
print("  - Upload image to current directory first")
print("")
print("CELL 9: Evaluate Model (Optional)")
print("  - Evaluates model on test set")
print("  - Generates evaluation metrics")
print("")
print("CELL 10: Save Results (Optional)")
print("  - Copies checkpoints and results to persistent location")
print("  - Saves to $HOME/hieroglyph_classifier_results")
print("")
print("CELL 11: Cleanup (Optional)")
print("  - Removes old checkpoints")
print("  - Cleans temporary files")
print("  - Frees disk space")
print("")
print("="*80)
print("QUICK START:")
print("="*80)
print("1. Run cells 1-4 sequentially (setup)")
print("2. Run cell 5 to preload data into RAM")
print("3. Run cell 6 to verify everything is ready")
print("4. Run cell 7 to start training")
print("5. Training will use RAM cache automatically (faster)")
print("6. Use cells 8-10 for inference/evaluation/saving")
print("")
print("MEMORY OPTIMIZATION:")
print("  - 64GB RAM: All training data loaded into RAM")
print("  - 48GB VRAM (RTX 8000): Large batch sizes (64)")
print("  - No disk I/O during training = much faster")
print("")
print("TROUBLESHOOTING:")
print("  - Low disk space: Notebook auto-filters datasets")
print("  - RAM issues: Set USE_RAM_CACHE = False in cell 5")
print("  - GPU not detected: Check CUDA installation")
print("="*80)
